In [ ]:
from optparse import OptionParser
import sys
import pickle
from itertools import islice, chain
import gzip
from collections import defaultdict
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

import regex as re

import Levenshtein

In [ ]:
#inputs for Guide RNA quantification
bcs = np.loadtxt("cb.txt", dtype=str) #load cell barcodes extracted from seurat object built on RNA
guides = np.loadtxt("guides.txt",  dtype=str) #load guides 
guide_seq = guides[:,1] #extract sequences of each guides
guide_seq = np.char.upper(guide_seq) #convert sequences to upper case
guides = guides[:,0] #extract guide name
print(guide_seq)
print(guides)
guide_cr = "GCTATGCTGTTTCCAGCTTAGCTCTTAAAC" #constant region to look for
umi_n_muts = 1 #how many mutations to allow when classifying a read as a unique molecule

r1_file = "R1.fastq.gz" #put path to your R1 file here
r2_file = "R2.fastq.gz"  #put path to your R2 file here

guide_length = len(guide_seq[1])
print(guide_length)

In [ ]:
# hamming distance function
# we use this to assign CB/UMI, as well as guide identity
def _hamming(s1, s2) :
    d = 0.
    for j in range(len(s1)) :
        if s1[j] != s2[j] :
             d += 1.
    return d

In [ ]:
#Build dictionary of cell barcodes (single-mutation support)
#the keys are equal to all cell barcodes (and their 1bp mutants, assuming this doesn't collide with 
#another real barcode
bases = ['A', 'C', 'G', 'T'] 
barcode_dict = {}
sequences = []
for i in range(len(bcs)):
    bc = bcs[i]
    barcode_dict[bc]  = i
    for pos1 in range(len(bc)) :
        for b1 in bases:
            bc_mut = bc[:pos1] + b1 + bc[pos1+1:]
            if bc_mut in barcode_dict and barcode_dict[bc_mut] != i :
                del barcode_dict[bc_mut] 
            else:
                barcode_dict[bc_mut] = i

In [ ]:
#build ALT2 dictionary of guides
#now only the original guide sequences are used as keys
gdict = {}
sequences = []
#for i in range(len(guide_seq)):
for i, g in enumerate(guide_seq):
    # Directly map the original guide sequence to its index
    g = guide_seq[i]
    gdict[g] = i

In [ ]:
#initialize CB x guides matrix
guide_m = np.zeros((len(bcs), len(guide_seq))) 

#make dict to keep track of what umis have been seen for each CB
umi_dict = {}
for bc in bcs:
    umi_dict[bc] = {}
    
r1 = gzip.open(r1_file)
r2 = gzip.open(r2_file)

#initialize umi counting
total_line = 0
skips = 0 
not_skips = 0
guide_reads = 0

#while total_line < 100000: 
# recommend this for debugging if you want to just run a subset of lines
while True :  #comment this out if you dont' want to process the entire file
    read_name = r2.readline()
    if len(read_name) == 0: 
        break
    seq = r2.readline().strip()
    seq = seq.decode('utf8')
    strand = r2.readline().strip()
    quality = r2.readline().strip()

    read1_name = r1.readline().strip()
    read1_seq = r1.readline().strip()
    read1_seq = read1_seq.decode('utf8')
    read1_strand = r1.readline().strip()
    read1_quality = r1.readline().strip()

    total_line += 1
    if total_line % 1000000 == 0 :
        print("Processing read " + str(total_line) + "...")

    #check the hamming distance between the constant region and the beginning of the read
    hamming_dist = _hamming(seq[0:len(guide_cr)], guide_cr)
    if hamming_dist >3:
        continue
    else: 
        guide_reads +=1
        gseq = seq[len(guide_cr):len(guide_cr) + guide_length]
        #print(guide_seq)
        #print(len(guide_seq))

        #now check which guide matches sequence
        if gseq in gdict:
            g = guides[gdict[gseq]] #get guide identity allowing 1bp mismatch
            #correct cell barcode
            cb = read1_seq[0:16] 
            if cb in barcode_dict:
                cb_correct = bcs[barcode_dict[cb]]
                #now check UMI
                umi = read1_seq[16:16+12] 
                #add the guide to the umi dictionary
                if g not in umi_dict[cb_correct] :
                    umi_dict[cb_correct][g] = {}
                
                #increment the count if that UMI has been seen for that barcode
                umi_visited = False
                if umi in umi_dict[cb_correct][g]: 
                    umi_visited = True
                    umi_dict[cb_correct][g][umi] += 1
                #now check mutations       
                if umi_n_muts == 1 and not umi_visited:
                     for pos1 in range(len(umi)):
                        for b1 in bases :
                            umi_mut = umi[:pos1] + b1 + umi[pos1+1:]
                            if umi_mut in umi_dict[cb_correct][g] :
                                umi_visited = True
                                umi_dict[cb_correct][g][umi_mut] += 1
                                break #don't check every other UMI mutant
                        if umi_visited : 
                            break #don't check every other UMI mutant

                #if we've encounted this UMI before for this guide/cell, 
                #then we skip this read
                if umi_visited:  #count number of times we skip a read 
                    skips += 1
                    continue  
                #if it's a new UMI, increment count matrix appropriately
                else: 
                    not_skips += 1
                    umi_dict[cb_correct][g][umi] = 1
                    guide_m[barcode_dict[cb], gdict[gseq]] += 1.
        else:
            continue


In [ ]:
print(guide_m.sum())
#print(guide_reads)

In [ ]:
#save guide matrix
np.savetxt('guide_counts' + '.txt', guide_m, fmt='%s')

In [ ]:
#inputs for ADT quantification - since this uses the same script, i kept the variable names the same even though it is ADTs not guides
bcs = np.loadtxt("cb.txt", dtype=str) #load cell barcodes 
guides = np.loadtxt("adt.txt",  dtype=str) #load guides 
guide_seq = guides[:,1] #extract sequences of each guides
guide_seq = np.char.upper(guide_seq) #convert sequences to upper case
guides = guides[:,0] #extract guide name
print(guide_seq)
print(guides)
guide_cr = "GCTTTAAGGCCGGTCC" #constant region to look for
umi_n_muts = 1 #how many mutations to allow when classifying a read as a unique molecule


r1_file = "R1.fastq.gz" #put path to your R1 file here
r2_file = "R2.fastq.gz"  #put path to your R2 file here



In [ ]:
#Build dictionary of cell barcodes (single-mutation support)
#the keys are equal to all cell barcodes (and their 1bp mutants, assuming this doesn't collide with 
#another real barcode
bases = ['A', 'C', 'G', 'T'] 
barcode_dict = {}
sequences = []
for i in range(len(bcs)):
    bc = bcs[i]
    barcode_dict[bc]  = i
    for pos1 in range(len(bc)) :
        for b1 in bases:
            bc_mut = bc[:pos1] + b1 + bc[pos1+1:]
            if bc_mut in barcode_dict and barcode_dict[bc_mut] != i :
                del barcode_dict[bc_mut] 
            else:
                barcode_dict[bc_mut] = i

In [ ]:
#build dictionary of guides
#similar structure where the keys are all sequences within 1bp edits of guide, 
#and values correspond to index of original guide sequence
gdict = {}
sequences = []
#for i in range(1):
for i in range(len(guide_seq)):
    g = guide_seq[i]
    gdict[g] = i
    for pos1 in range(len(g)):
        for b1 in bases:
            g_mut =  g[:pos1] + b1 + g[pos1+1:]
            if g_mut in gdict and gdict[g_mut] != i:
                print("[ERROR] Barcode dictionary collision.")
                continue
            else: 
                #print(g_mut)
                gdict[g_mut] = i
                #print(g_mut)
                
            

In [ ]:
#initialize CB x guides matrix
guide_m = np.zeros((len(bcs), len(guide_seq))) 

#make dict to keep track of what umis have been seen for each CB
umi_dict = {}
for bc in bcs:
    umi_dict[bc] = {}
    
r1 = gzip.open(r1_file)
r2 = gzip.open(r2_file)

#initialize umi counting
total_line = 0
skips = 0 
not_skips = 0
guide_reads = 0

#while total_line < 100000: 
# recommend this for debugging if you want to just run a subset of lines
while True :  #comment this out if you dont' want to process the entire file
    read_name = r2.readline()
    if len(read_name) == 0: 
        break
    seq = r2.readline().strip()
    seq = seq.decode('utf8')
    strand = r2.readline().strip()
    quality = r2.readline().strip()

    read1_name = r1.readline().strip()
    read1_seq = r1.readline().strip()
    read1_seq = read1_seq.decode('utf8')
    read1_strand = r1.readline().strip()
    read1_quality = r1.readline().strip()

    total_line += 1
    if total_line % 1000000 == 0 :
        print("Processing read " + str(total_line) + "...")

    #check the hamming distance between the constant region and the beginning of the read
    hamming_dist = _hamming(seq[34:50], guide_cr)
    if hamming_dist >3:
        continue
    else: 
        guide_reads +=1
        gseq = seq[10:25]
        #print(guide_seq)
        #print(len(guide_seq))

        #now check which guide matches sequence
        if gseq in gdict:
            g = guides[gdict[gseq]] #get guide identity allowing 1bp mismatch
            #correct cell barcode
            cb = read1_seq[0:16] 
            if cb in barcode_dict:
                cb_correct = bcs[barcode_dict[cb]]
                #now check UMI
                umi = read1_seq[16:16+12] #lol i hard-coded this
                #add the guide to the umi dictionary
                if g not in umi_dict[cb_correct] :
                    umi_dict[cb_correct][g] = {}
                
                #increment the count if that UMI has been seen for that barcode
                umi_visited = False
                if umi in umi_dict[cb_correct][g]: 
                    umi_visited = True
                    umi_dict[cb_correct][g][umi] += 1
                #now check mutations       
                if umi_n_muts == 1 and not umi_visited:
                     for pos1 in range(len(umi)):
                        for b1 in bases :
                            umi_mut = umi[:pos1] + b1 + umi[pos1+1:]
                            if umi_mut in umi_dict[cb_correct][g] :
                                umi_visited = True
                                umi_dict[cb_correct][g][umi_mut] += 1
                                break #don't check every other UMI mutant
                        if umi_visited : 
                            break #don't check every other UMI mutant

                #if we've encounted this UMI before for this guide/cell, 
                #then we skip this read
                if umi_visited:  #count number of times we skip a read 
                    skips += 1
                    continue  
                #if it's a new UMI, increment count matrix appropriately
                else: 
                    not_skips += 1
                    umi_dict[cb_correct][g][umi] = 1
                    guide_m[barcode_dict[cb], gdict[gseq]] += 1.
        else:
            continue

In [ ]:
print(guide_m.sum())
#print(guide_reads)

In [ ]:
#save ADT matrix
np.savetxt('adt_counts' + '.txt', guide_m, fmt='%s')


In [ ]:
#inputs for OligoHash Quantification - again, the variables are the same names, but we have to reenter them so they point to the write place
bcs = np.loadtxt("cb.txt", dtype=str) #load cell barcodes 
guides = np.loadtxt("oligohash.txt",  dtype=str) #load guides 
guide_seq = guides[:,1] #extract sequences of each guides
guide_seq = np.char.upper(guide_seq) #convert sequences to upper case
guides = guides[:,0] #extract guide name
print(guide_seq)
print(guides)
guide_cr = "GCTTTAAGGCCGGTCCTAGCAA" #constant region to look for
umi_n_muts = 1 #how many mutations to allow when classifying a read as a unique molecule

r1_file = "R1.fastq.gz" #put path to your R1 file here
r2_file = "R2.fastq.gz" #put path to your R2 file here


In [ ]:
#Build dictionary of cell barcodes (single-mutation support)
#the keys are equal to all cell barcodes (and their 1bp mutants, assuming this doesn't collide with 
#another real barcode
bases = ['A', 'C', 'G', 'T'] 
barcode_dict = {}
sequences = []
for i in range(len(bcs)):
    bc = bcs[i]
    barcode_dict[bc]  = i
    for pos1 in range(len(bc)) :
        for b1 in bases:
            bc_mut = bc[:pos1] + b1 + bc[pos1+1:]
            if bc_mut in barcode_dict and barcode_dict[bc_mut] != i :
                del barcode_dict[bc_mut] 
            else:
                barcode_dict[bc_mut] = i

In [ ]:
#build dictionary of guides
#similar structure where the keys are all sequences within 1bp edits of guide, 
#and values correspond to index of original guide sequence
gdict = {}
sequences = []
#for i in range(1):
for i in range(len(guide_seq)):
    g = guide_seq[i]
    gdict[g] = i
    for pos1 in range(len(g)):
        for b1 in bases:
            g_mut =  g[:pos1] + b1 + g[pos1+1:]
            if g_mut in gdict and gdict[g_mut] != i:
                print("[ERROR] Barcode dictionary collision.")
                continue
            else: 
                #print(g_mut)
                gdict[g_mut] = i
                #print(g_mut)
                

In [ ]:
#initialize CB x guides matrix
guide_m = np.zeros((len(bcs), len(guide_seq))) 

#make dict to keep track of what umis have been seen for each CB
umi_dict = {}
for bc in bcs:
    umi_dict[bc] = {}
    
r1 = gzip.open(r1_file)
r2 = gzip.open(r2_file)

#initialize umi counting
total_line = 0
skips = 0 
not_skips = 0
guide_reads = 0

# while total_line < 100000: 
# recommend this for debugging if you want to just run a subset of lines
while True :  #comment this out if you dont' want to process the entire file
    read_name = r2.readline()
    if len(read_name) == 0: 
        break
    seq = r2.readline().strip()
    seq = seq.decode('utf8')
    strand = r2.readline().strip()
    quality = r2.readline().strip()

    read1_name = r1.readline().strip()
    read1_seq = r1.readline().strip()
    read1_seq = read1_seq.decode('utf8')
    read1_strand = r1.readline().strip()
    read1_quality = r1.readline().strip()

    total_line += 1
    if total_line % 1000000 == 0 :
        print("Processing read " + str(total_line) + "...")

    #check the hamming distance between the constant region and the beginning of the read
    hamming_dist = _hamming(seq[15:37],guide_cr)
    if hamming_dist >3:
        continue
    else: 
        guide_reads +=1
        gseq = seq[0:15]
        #print(guide_seq)
        #print(len(guide_seq))

        #now check which guide matches sequence
        if gseq in gdict:
            g = guides[gdict[gseq]] #get guide identity allowing 1bp mismatch
            #correct cell barcode
            cb = read1_seq[0:16] 
            if cb in barcode_dict:
                cb_correct = bcs[barcode_dict[cb]]
                #now check UMI
                umi = read1_seq[16:16+12] 
                #add the guide to the umi dictionary
                if g not in umi_dict[cb_correct] :
                    umi_dict[cb_correct][g] = {}
                
                #increment the count if that UMI has been seen for that barcode
                umi_visited = False
                if umi in umi_dict[cb_correct][g]: 
                    umi_visited = True
                    umi_dict[cb_correct][g][umi] += 1
                #now check mutations       
                if umi_n_muts == 1 and not umi_visited:
                     for pos1 in range(len(umi)):
                        for b1 in bases :
                            umi_mut = umi[:pos1] + b1 + umi[pos1+1:]
                            if umi_mut in umi_dict[cb_correct][g] :
                                umi_visited = True
                                umi_dict[cb_correct][g][umi_mut] += 1
                                break #don't check every other UMI mutant
                        if umi_visited : 
                            break #don't check every other UMI mutant

                #if we've encounted this UMI before for this guide/cell, 
                #then we skip this read
                if umi_visited:  #count number of times we skip a read 
                    skips += 1
                    continue  
                #if it's a new UMI, increment count matrix appropriately
                else: 
                    not_skips += 1
                    umi_dict[cb_correct][g][umi] = 1
                    guide_m[barcode_dict[cb], gdict[gseq]] += 1.
        else:
            continue


In [ ]:
print(guide_m.sum())
#print(guide_reads)

In [ ]:
#save OligoHash matrix
np.savetxt('OligoHash_counts'+'.txt', guide_m, fmt='%s')